# Analyze Google Play Store App Reviews using Snowflake Cortex LLMs

In [ ]:
# Python Packages
import streamlit as st
import pandas as pd
from google_play_scraper import Sort, reviews_all
import warnings
warnings.filterwarnings("ignore")

# Snowpark Packages
from snowflake.cortex import Complete
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import col, lit
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T

# Create Snowpark Session
session = get_active_session()

## Retrieve Google Playstore Reviews

In [ ]:
appid = 'com.emirates.ek.android'

reviews = reviews_all(
    appid,
    sleep_milliseconds=0, # defaults to 0
    lang='en', # defaults to 'en'
    country='us', # defaults to 'us'
    sort=Sort.NEWEST, # defaults to Sort.MOST_RELEVANT
)

df = pd.DataFrame(reviews)
df.columns = [col.upper() for col in df.columns]

## Persist reviews in Snowflake table

In [ ]:
app_reviews = session.write_pandas(df, table_name='EMIRATES_APP_REVIEWS', overwrite=True, auto_create_table=True)
# filter reviews with low rating and enough content
app_reviews = app_reviews.filter(col('score') < 2).filter(F.length('CONTENT') > 200)

print('Numer of reviews:', app_reviews.count())
app_reviews

## Analyze Reviews with Cortex

In [ ]:
llm_model = 'llama3-8b'

# Create a prompt
prompt = F.concat(lit("Derive the sentiment (1-5) and assign the review to one or multiple categories. \
Categories: baggage_lost, customer_service, login, app_crash, refund.\
The review: "), col('CONTENT'), lit("Return a JSON like this {sentiment:sentiment, categories:[category1, category2, category3,]}.\
Only return the JSON, no other text."))

# Run LLM and parse outputs
app_reviews = app_reviews.with_column('LLM_OUTPUT', F.call_builtin('try_parse_json',Complete(llm_model,prompt))).cache_result()
app_reviews = app_reviews.filter(col('LLM_OUTPUT').is_not_null())
app_reviews = app_reviews.with_column('LLM_SENTIMENT', col('LLM_OUTPUT')['sentiment'].cast('int'))
app_reviews = app_reviews.with_column('LLM_CATEGORIES', col('LLM_OUTPUT')['categories'].cast('array'))
app_reviews = app_reviews.join_table_function('flatten', col('LLM_CATEGORIES'))
app_reviews = app_reviews.with_column('LLM_CATEGORY_INDEX', col('INDEX').cast('int'))
app_reviews = app_reviews.with_column('LLM_CATEGORY', col('VALUE').cast('string'))
app_reviews = app_reviews[['CONTENT','SCORE','LLM_SENTIMENT','LLM_CATEGORY_INDEX','LLM_CATEGORY']]
app_reviews.show()

In [ ]:
plot_df = app_reviews.group_by(['LLM_SENTIMENT','LLM_CATEGORY']).agg(F.count('LLM_CATEGORY').as_('COUNT'))
st.subheader('Emirates App Reviews by category')
st.bar_chart(plot_df, x='LLM_SENTIMENT', y='COUNT', color='LLM_CATEGORY')

In [ ]:
llm_model = 'llama3-70b'

# Create a prompt
prompt = F.concat(lit("You are given an app review and a reply from the app developer. \
Rate the reply on a scale 1-5.\
Return a JSON like this {rating:rating, rating_description:rating_description, suggested_reply:suggested_reply}.\
Only return the JSON, no other text. The review: "), col('CONTENT'), lit(' The reply: '), col('REPLYCONTENT'))

developer_responses = session.table('EMIRATES_APP_REVIEWS')
developer_responses = developer_responses.filter(col('SCORE') <2).filter(col('REPLYCONTENT').is_not_null())
developer_responses = developer_responses.with_column('LLM_OUTPUT', F.call_builtin('try_parse_json',Complete(llm_model,prompt))).cache_result()
developer_responses = developer_responses.filter(col('LLM_OUTPUT').is_not_null())
developer_responses = developer_responses.with_column('REPLY_RATING', col('LLM_OUTPUT')['rating'].cast('int'))
developer_responses = developer_responses.with_column('RATING_DESCRIPTION', col('LLM_OUTPUT')['rating_description'].cast('string'))
developer_responses = developer_responses.with_column('SUGGESTED_REPLY', col('LLM_OUTPUT')['suggested_reply'].cast('string'))
developer_responses = developer_responses.order_by(col('REPLY_RATING'), ascending=True)
developer_responses = developer_responses[['CONTENT','REPLYCONTENT','REPLY_RATING','RATING_DESCRIPTION','SUGGESTED_REPLY']]
developer_responses

In [ ]:
plot_df = developer_responses.group_by('REPLY_RATING').agg(F.count('REPLY_RATING').as_('COUNT'))
st.subheader('Emirates App Developer Responses by LLM Rating')
st.bar_chart(plot_df, x='REPLY_RATING', y='COUNT')